# Import Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import tensorflow as tf
import tensorflow.keras as keras

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
housing_data = fetch_california_housing()
X = housing_data.data
y = housing_data.target

In [4]:
print(X[1:3])
print(y[1:3])

[[ 8.30140000e+00  2.10000000e+01  6.23813708e+00  9.71880492e-01
   2.40100000e+03  2.10984183e+00  3.78600000e+01 -1.22220000e+02]
 [ 7.25740000e+00  5.20000000e+01  8.28813559e+00  1.07344633e+00
   4.96000000e+02  2.80225989e+00  3.78500000e+01 -1.22240000e+02]]
[3.585 3.521]


In [5]:
housing_data.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

# Split train test

In [6]:
X_temp, X_test,y_temp,y_test = train_test_split(
    X,
    y,
    random_state=2024,
    test_size=0.2
)

X_train, X_valid,y_train, y_valid = train_test_split(
    X_temp,
    y_temp,
    random_state=2024,
    test_size=0.2
) 

# Train model

In [7]:
tf.random.set_seed(42)
norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])
model = tf.keras.Sequential([
    norm_layer,
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(1)
])
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])

In [8]:
# calculate mean and std

norm_layer.adapt(X_train)

## Tensorboard demo

In [10]:
import shutil

import keras_tuner
shutil.rmtree("my_logs", ignore_errors=True)

In [11]:
from pathlib import Path
from time import strftime

def get_run_logdir(root_logdir="my_logs"):
    return Path(root_logdir) / strftime("run_%Y_%m_%d_%H_%M_%S")

run_logdir = get_run_logdir()

## Fit the model

In [13]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir, profile_batch=(100, 200))
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid), callbacks = [tensorboard_cb])

Epoch 1/20
413/413 [==============================] - 2s 3ms/step - loss: 0.8328 - root_mean_squared_error: 0.9126 - val_loss: 0.4430 - val_root_mean_squared_error: 0.6655
Epoch 2/20
413/413 [==============================] - 1s 2ms/step - loss: 0.3948 - root_mean_squared_error: 0.6283 - val_loss: 0.3976 - val_root_mean_squared_error: 0.6306
Epoch 3/20
413/413 [==============================] - 1s 2ms/step - loss: 0.3645 - root_mean_squared_error: 0.6037 - val_loss: 0.3719 - val_root_mean_squared_error: 0.6098
Epoch 4/20
413/413 [==============================] - 1s 2ms/step - loss: 0.3447 - root_mean_squared_error: 0.5871 - val_loss: 0.3538 - val_root_mean_squared_error: 0.5948
Epoch 5/20
413/413 [==============================] - 1s 2ms/step - loss: 0.3258 - root_mean_squared_error: 0.5707 - val_loss: 0.3601 - val_root_mean_squared_error: 0.6001
Epoch 6/20
413/413 [==============================] - 1s 2ms/step - loss: 0.3143 - root_mean_squared_error: 0.5606 - val_loss: 0.3450 - val_

In [14]:
print("my_logs")
for path in sorted(Path("my_logs").glob("**/*")):
    print("  " * (len(path.parts) - 1) + path.parts[-1])

my_logs
  run_2024_04_19_21_53_52
    plugins
      profile
        2024_04_19_16_24_44
          LAPTOP-SRAMQVR5.xplane.pb
    train
      events.out.tfevents.1713543883.LAPTOP-SRAMQVR5.35184.0.v2
    validation
      events.out.tfevents.1713543885.LAPTOP-SRAMQVR5.35184.1.v2


In [17]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs

from IPython.display import display, HTML
display(HTML('<a href="http://localhost:6006/">http://localhost:6006/</a>'))

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 29124), started 0:01:24 ago. (Use '!kill 29124' to kill it.)

In [18]:
from tensorboard import notebook

notebook.list()

Known TensorBoard instances:
  - port 6006: logdir logs/fit (started 10:28:58 ago; pid 25480)
  - port 6006: logdir ./my_logs (started 0:02:15 ago; pid 29124)
